In [1]:
import subprocess

In [2]:
prepositions = ['в', 'без', 'до', 'из', 'к', 'на', 'по', 'о', 'от', 'перед', 'при', 'через',
                'с', 'у', 'и', 'за', 'над', 'для', 'об', 'под', 'про', 'то', 'это', 'а']

speech_parts = ['S', 'A', 'V', 'ADV']

In [3]:
def divide_in_sentences(text):
    sentences = list()
    sentences = text.split('. ')
    return sentences

In [7]:
def mystem_func(sentence):
    """Возвращает список, состоящий из списков вида [token, lemma, pos]"""
    
    # Удаляем посторонние символы
    table = {ord(char): None for char in '!?,.;'}  
    sentence = sentence.translate(table)
    
    # Выполнение команды mystem
    command = "echo '%s' | mystem -e cp866 -nid" % (sentence)  
    proc = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, )
    output = proc.communicate()[0] 
    
    # Обработка результата в считываемый вид (массив строк)
    string_output = str(output.decode('cp866'))  
    output_array = string_output.split('\r\n')
    output_array.remove('')
    
    # Обработка каждой строки так, чтобы получить для слова его токен, лемму и часть речи.
    # Результат для каждого слова записывается в список из трех элементов.
    # Все списки хранятся в списке words_array
    words_array = list()
    for word_line in output_array:
        start_index = word_line.find('{',)
        end_index = len(word_line) - 1
        token = word_line[:start_index]
        if (token.lower() in prepositions):  #  Если исходное слово - предлог, то выбрасываем и идем дальше.
            continue
        
        # Отбрасываем лишнюю информацию
        buf = word_line[start_index+1:end_index].split(',')
        buf = buf[0].split('=')
        if (buf[1] not in speech_parts):
            continue
        if (len(buf) > 2):
            del buf[2]
        
        # Оформляем результат в виде списка
        buf.insert(0, token)
        words_array.append(buf)
    return words_array

In [5]:
text = (
    'Если подросток добрый, уступчивый и хороший, '
    + 'то это часто воспринимается окружающими как проявление слабости его характера, '
    + 'как неспособность чётко высказать свою позицию. '
    + 'Привет, незнакомка.'
)

In [11]:
sentences = divide_in_sentences(text)
result = list()
for sentence in sentences:
    result_sentence = list()
    result_sentence = mystem_func(sentence)
    result.append(result_sentence)
for i in range(0,len(result)):
    print('\033[1m' + 'Предложение %i:'%i + '\033[0m', result[i])

Предложение 0: [['подросток', 'подросток', 'S'], ['добрый', 'добрый', 'A'], ['уступчивый', 'уступчивый', 'A'], ['хороший', 'хороший', 'A'], ['часто', 'часто', 'ADV'], ['воспринимается', 'восприниматься', 'V'], ['окружающими', 'окружающие', 'S'], ['проявление', 'проявление', 'S'], ['слабости', 'слабость', 'S'], ['характера', 'характер', 'S'], ['неспособность', 'неспособность', 'S'], ['чётко', 'четко', 'ADV'], ['высказать', 'высказывать', 'V'], ['позицию', 'позиция', 'S']]
Предложение 1: [['Привет', 'привет', 'S'], ['незнакомка', 'незнакомка', 'S']]
